In [9]:
import transformers
import torch
import accelerate

In [4]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [10]:
pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
pipeline("Hey how are you doing today?")